In [3]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor 

pre-process

In [2]:
df = pd.read_csv(r'./Data/processed.csv')
X = df.drop(['KWH'], axis=1)
Y = df['KWH']

In [4]:
best_df = SelectKBest(mutual_info_regression, k = 100).fit_transform(X, Y)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(best_df, Y, test_size=0.3, shuffle = True, random_state = 42)

In [6]:
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

models implementation

In [7]:
reg = LinearRegression().fit(X_train, Y_train)
print(reg.score(X_test, Y_test))
print(reg.score(X_train, Y_train))

print("\n\n")

lasso = Lasso(alpha=0.01)
lasso.fit(X_train, Y_train)
print(lasso.score(X_test, Y_test))
print(lasso.score(X_train, Y_train))

0.9858103000107591
0.9866861993141803



0.9857917397069654
0.9866781204120209
C:\Users\Daniel Ciovica\miniconda3\envs\bigdata\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593647246.4896336, tolerance: 51375337.476005666
  model = cd_fast.enet_coordinate_descent(


In [8]:
sgd = SGDRegressor(
    random_state = 42,
    shuffle= True, 
    early_stopping= True,
    validation_fraction= 0.2 
)

sgd.fit(X_train, Y_train)
print("Score: {}".format(sgd.score(X_test, Y_test)))


Score: 0.8246414771458046


In [ ]:
# sgd = SGDRegressor(
#     loss = 'squared_loss',
#     penalty = 'l1',
#     alpha= 0.001,
#     max_iter = 1000,
#     tol = 1e-4,
#     shuffle= True,
#     verbose = 0,
#     random_state = 42,
#     learning_rate = 'adaptive',
#     early_stopping= True ,
#     validation_fraction= 0.2,
# )

In [13]:
rr = Ridge(
    random_state=42,
    solver = 'auto'

)
rf = RandomForestRegressor(
    oob_score = True, 
    n_jobs = -1, 
    random_state = 42,
)
sgd = SGDRegressor( 
    random_state=42,
    penalty='l1', 
)
ls = Lasso(
    random_state = 42 
)
estimators = [rr, rf, sgd, ls]

In [14]:
param_rr = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1]} 
param_rf = {'n_estimators': [5, 50, 100, 200, 500], 'max_depth': [1, 5, 10, 15], 'min_samples_leaf': [1, 5, 10]}
param_sgd = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1], 'loss': ['squared_epsilon_insensitive', 'huber'], 'learning_rate': ['optimal', 'adaptive']}
param_ls = {"selection":["cyclic", "random"], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}
 
 
params = [param_rr, param_rf, param_sgd, param_ls]

In [17]:
def performingGridSearch(estimators, X_train, Y_train, X_test, Y_test, params):
  scores_train = []
  scores_test = []
 
  for index, estimator in enumerate(estimators):
    model = GridSearchCV(estimator, param_grid = params[index], cv = 3, n_jobs=1, scoring='r2', verbose = 3)
    model.fit(X_train, Y_train)
    best_params = model.best_params_
    scores_train.append(model.score(X_train, Y_train))
 
    print("\n\n Model: {} \n\n Best params: {} ".format(estimator, best_params))
    scores_test.append(model.score(X_test, Y_test))
    
    
    
  return scores_train, scores_test

In [18]:
scores_train, scores_val = performingGridSearch(estimators, X_train, Y_train, X_test, Y_test, params)

kages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\Daniel Ciovica\miniconda3\envs\bigdata\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "C:\Users\Daniel Ciovica\miniconda3\envs\bigdata\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    future = self._workers.submit(SafeFunction(func))
  File "C:\Users\Daniel Ciovica\miniconda3\envs\bigdata\lib\site-packages\joblib\externals\loky\reusable_executor.py", line 177, in submit
    return super(_ReusablePoolExecutor, self).submit(
  File "C:\Users\Daniel Ciovica\miniconda3\envs\bigdata\lib\site-packages\joblib\externals\loky\process_executor.py", line 1102, in submit
    raise self._flags.broken
joblib.externals.loky.process_executor.TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the f

KeyboardInterrupt: 

In [10]:
# best_params = model.best_params_
# print("\n\n Model: {} \n\n Best params: {} ".format(model, best_params))



 Model: GridSearchCV(cv=5,
             estimator=SGDRegressor(early_stopping=True, random_state=42,
                                    validation_fraction=0.2),
             n_jobs=1,
             param_grid={'alpha': [1, 0.1, 0.01, 0.001, 0.001, 0.0001, 1e-05],
                         'learning_rate': ['optimal', 'invscaling', 'adaptive'],
                         'loss': ['squared_loss', 'huber',
                                  'epsilon_insensitive',
                                  'squared_epsilon_insensitive'],
                         'penalty': ['l1', 'l2'],
                         'tol': [0.001, 0.0001, 1e-05]},
             scoring='r2', verbose=3) 

 Best params: {'alpha': 1e-05, 'learning_rate': 'adaptive', 'loss': 'squared_epsilon_insensitive', 'penalty': 'l1', 'tol': 0.001} 


In [11]:
sgd = SGDRegressor(
    random_state = 42,
    shuffle= True, 
    early_stopping= True,
    validation_fraction= 0.2,
    loss = 'squared_epsilon_insensitive',
    penalty = 'l1',
    alpha = 1e-05,
    learning_rate = 'adaptive',
    tol = 0.001, 

)
sgd.fit(X_train, y_train)

SGDRegressor(alpha=1e-05, early_stopping=True, learning_rate='adaptive',
             loss='squared_epsilon_insensitive', penalty='l1', random_state=42,
             validation_fraction=0.2)

In [13]:
print("\n\n Train summary: \n\n")
print("\n\n R2 Score: {} ".format(model.score(X_train, y_train)))

y_train_pred = sgd.predict(X_train)
print("\n\n MSE Score: {} ".format(mean_squared_error(y_train, y_train_pred)))
print("\n\n MAE Score: {} ".format(mean_absolute_error(y_train, y_train_pred)))

print("\n\n Test summary: \n\n")
y_test_pred = sgd.predict(X_test)

print("\n\n R2 Test Score: {} ".format(model.score(X_test, y_test)))

print("\n\n MSE Score: {} ".format(mean_squared_error(y_test, y_test_pred)))
print("\n\n MAE Score: {} ".format(mean_absolute_error(y_test, y_test_pred)))



 Train summary: 




 R2 Score: 0.9499688690451299 


 MSE Score: 3039695.1716091353 


 MAE Score: 1124.2350642041351 


 Test summary: 




 R2 Test Score: 0.9503455658056125 


 MSE Score: 2619006.1415181463 


 MAE Score: 1085.7797531133097 


In [14]:
print(y_test_pred[:10])
print(list(y_test[:10]))

[13233.9367092   2708.7482805  27548.10541351 23462.20094868
  3662.84483576 10571.55515894  3285.0298734  20020.33122668
  1400.71838817 14443.87892858]
[13796, 2467, 29209, 24099, 4501, 10469, 3997, 21627, 3101, 13540]


In [19]:
df = pd.read_csv(r'./Data/processed.csv')
X = df.drop(['KWH'], axis=1)
y = df['KWH']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, shuffle = True, random_state = 42)

scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


sgd = SGDRegressor(
    random_state = 42,
    shuffle= True, 
    early_stopping= True,
    validation_fraction= 0.2,
    loss = 'squared_epsilon_insensitive',
    penalty = 'l1',
    alpha = 1e-05,
    learning_rate = 'adaptive',
    tol = 0.001, 
)

sgd.fit(X_train, Y_train)
print("Score: {}".format(sgd.score(X_test, Y_test)))

Score: 0.9515730125936807


In [21]:
print("\n\n Train summary: \n\n")
print("\n\n R2 Score: {} ".format(sgd.score(X_train, Y_train)))

y_train_pred = sgd.predict(X_train)
print("\n\n MSE Score: {} ".format(mean_squared_error(Y_train, y_train_pred)))
print("\n\n MAE Score: {} ".format(mean_absolute_error(Y_train, y_train_pred)))

print("\n\n Test summary: \n\n")
y_test_pred = sgd.predict(X_test)

print("\n\n R2 Test Score: {} ".format(sgd.score(X_test, y_test)))

print("\n\n MSE Score: {} ".format(mean_squared_error(Y_test, y_test_pred)))
print("\n\n MAE Score: {} ".format(mean_absolute_error(Y_test, y_test_pred)))



 Train summary: 




 R2 Score: 0.9599462478664899 


 MSE Score: 2433508.7902546925 


 MAE Score: 1048.8571534582022 


 Test summary: 




 R2 Test Score: 0.9515730125936807 


 MSE Score: 2554264.8806721875 


 MAE Score: 1123.5811108095636 
